# Visualisation

## Du pixel aux images - 32M7138

*Printemps 2025 - Université de Genève*

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

## **Plan du cours**

> **Visuliser les images en quantité**
> * Avec une une boucle
> * Créer une mosaïque
> * Autres outils
> * Aller plus loin dans la visualisation des métadonnées

In [ ]:
# colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# librairies
import os
import pandas as pd
import requests
import hashlib
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

In [ ]:
# télécharger les images à partir d'un csv
csv_path = "38-vangogh.csv"
df = pd.read_csv(csv_path)
df = df.drop_duplicates()
df.head()

In [ ]:
df.info()

In [ ]:
# créer un nom de fichier, robuste

df["hash"] = [hashlib.sha1(str.encode(str(i))).hexdigest() for i in df["image_url"]]
df["filename"] = df["hash"] + ".jpg"
df.filename

In [ ]:
# télécharger les images

output_dir = "downloaded_images"
os.makedirs(output_dir, exist_ok=True)

for _, row in tqdm(df.iterrows()):
    try:
        url = row["image_url"]
        fname = os.path.join(output_dir, row["filename"])
        img_data = requests.get(url).content # télécharger
        with open(fname, 'wb') as handler: # définir le fichier et son chemin
            handler.write(img_data)
    except Exception as e:
        print(f"Erreur téléchargement {row['image_url']}: {e}")

In [ ]:
# supprimer les images non-téléchargées de notre csv

downloaded_images = os.listdir(output_dir)
df_clean = df[df["filename"].isin(downloaded_images)]
df_clean.info()

## Visuliser les images en quantité
### Avec une une boucle

In [ ]:
# nos variables pour la visualisation

print(len(os.listdir(output_dir)), "\n")
print(df_clean.info())

In [ ]:
# faire une plus petite dataframe 

sub_df_clean = df_clean[:10]
sub_df_clean.info()

In [ ]:
# visualiser

for _, row in sub_df_clean.iterrows():
    file_path = os.path.join(output_dir, row["filename"])
    image = cv2.imread(file_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    title = f"{row['Title']} - {row['Date']}"
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()    


In [ ]:
# visualiser et sauvegarder en boucle
out_dir = "visualisation_vanGogh"
os.makedirs(out_dir, exist_ok=True)

for _, row in sub_df_clean.iterrows():
    file_path = os.path.join(output_dir, row["filename"])
    image = cv2.imread(file_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    title = f"{row['Title']} - {row['Date']}"
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis("off")

    # Enregistrement (au format PNG par exemple)
    out = row["filename"] + "titleDate_visu.png"
    out_file = os.path.join(out_dir, out)
    plt.savefig(out_file, bbox_inches='tight')

### Créer une mosaïque

In [ ]:
# Paramètre de la grille
cols = 4  # nombre de colonnes
images = []
titles = []

# Charger les images et titres
for _, row in sub_df_clean.iterrows():
    file_path = os.path.join(output_dir, row["filename"])
    image = cv2.imread(file_path)
    if image is not None:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image_rgb)
        titles.append(f"{row['Title']} - {row['Date']}")

# Taille de la figure
rows = math.ceil(len(images) / cols)
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))

# Affichage
for idx in range(rows * cols):
    r, c = divmod(idx, cols)
    ax = axes[r][c] if rows > 1 else axes[c]  # gestion 1D/2D
    if idx < len(images):
        ax.imshow(images[idx])
        ax.set_title(titles[idx], fontsize=10)
        ax.axis("off")
    else:
        ax.axis("off")  # case vide

plt.tight_layout()
plt.show()


In [ ]:
# Paramètre de la grille
cols = 4  # nombre de colonnes
images = []
titles = []

# Charger les images et titres
for _, row in sub_df_clean.iterrows():
    file_path = os.path.join(output_dir, row["filename"])
    image = cv2.imread(file_path)
    if image is not None:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image_rgb)
        titles.append(f"{row['Title']} - {row['Date']}")

# Taille de la figure
rows = math.ceil(len(images) / cols)
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))

# Affichage
for idx in range(rows * cols):
    r, c = divmod(idx, cols)
    ax = axes[r][c] if rows > 1 else axes[c]  # gestion 1D/2D
    if idx < len(images):
        ax.imshow(images[idx])
        ax.set_title(titles[idx], fontsize=10)
        ax.axis("off")
    else:
        ax.axis("off")  # case vide

plt.tight_layout()
plt.savefig("mosaic_all_images.png", dpi=200)
plt.show()


### Autres outils

- https://collection-space-navigator.github.io
- https://dhlab.yale.edu/projects/pixplot/
- https://vikusviewer.fh-potsdam.de
- https://visual-computing.com/projects/picarrange
- https://github.com/damoncrockett/ivpy

### Pour aller plus loin dans la visualisation des métadonnées
- https://www.rawgraphs.io
- https://kepler.gl
- https://studio.foursquare.com
- https://flourish.studio